# 8.1 **ChatGPT**

## OpenAI API
https://platform.openai.com/

In [ ]:
!pip install -q cohere
!pip install -q openai
!pip install -q tiktoken

In [2]:
import openai
import tiktoken
from getpass import getpass

In [3]:
api_key = getpass('Input yout OpenAI API Key here: ')
openai.api_key = api_key

Input yout OpenAI API Key here: ··········



https://openai.com/pricing

**GPT-3.5-turbo**
- max 4,096 tokens (prompt + completion)
- Input: \$ 0.0015 / 1K tokens, Output: \$ 0.002 / 1K tokens

In [4]:
api_model = 'gpt-3.5-turbo'
encoding = tiktoken.encoding_for_model(api_model)

In [5]:
def get_num_input_tokens(content):
    num_template_tokens = 4 # <|start|>user\n{content}<|end|>
    num_priming_tokens = 3 # <|start|>assistant<|message|>
    num_tokens = len(encoding.encode(content))
    num_tokens += num_template_tokens + num_priming_tokens
    return num_tokens

In [6]:
def calc_price(num_input_tokens, num_output_tokens, input_unit=0.0015, output_unit=0.002):
    dollars = (num_input_tokens*input_unit + num_output_tokens*output_unit) / 1000
    return round(dollars, 5)

def estimate_price(num_input_tokens, input_to_output):
    estimated_output_tokens = num_input_tokens * input_to_output
    estimated_price = calc_price(num_input_tokens, estimated_output_tokens)
    return estimated_price

In [25]:
def query_message(message, temperature=0.1, verbose=False):
    exceeded = False
    messages = [{'role':'user', 'content': message}]

    response = openai.ChatCompletion.create(model=api_model, messages=messages, temperature=temperature)
    input_tokens = response['usage']['prompt_tokens']
    output_tokens = response['usage']['completion_tokens']
    if verbose:
        print('input tokens:', input_tokens)
        print('output tokens:', output_tokens)
        print('input to output ratio:', round(output_tokens / input_tokens, 3))
        print('price: $', calc_price(input_tokens, output_tokens))
    answer = response['choices'][0]['message']['content'].strip()

    return answer

## Data

FOMC statement: https://www.federalreserve.gov/newsevents/pressreleases/monetary20230920a.htm

FOMC minutes: https://www.federalreserve.gov/monetarypolicy/fomcminutes20230920.htm

In [8]:
import re
import requests
from bs4 import BeautifulSoup

In [9]:
def get_FOMC_text(date, text_type='statement'):
    if text_type == 'statement':
        url = 'https://www.federalreserve.gov/newsevents/pressreleases/monetary{}a.htm'.format(date)
    elif text_type == 'minutes':
        url = 'https://www.federalreserve.gov/monetarypolicy/fomcminutes{}.htm'.format(date)
    raw = requests.get(url)
    html = BeautifulSoup(raw.text, 'html.parser')
    text = html.find('div', id='article').get_text()
    return text

In [10]:
fomc_dates = ['20230201', '20230322', '20230503', '20230614', '20230726', '20230920']
statement_list = []
minutes_list = []
for fomc_date in fomc_dates:
    statement = get_FOMC_text(fomc_date, 'statement')
    statement = re.sub('\n+', '\n', statement)
    statement_list.append(statement)
    minutes = get_FOMC_text(fomc_date, 'minutes')
    minutes_list.append(minutes)

In [11]:
statement_list[-1]

"\nSeptember 20, 2023\nFederal Reserve issues FOMC statement\nFor release at 2:00 p.m. EDT                     \r\n                \r\n                \nShare\nRecent indicators suggest that economic activity has been expanding at a solid pace. Job gains have slowed in recent months but remain strong, and the unemployment rate has remained low. Inflation remains elevated.\nThe U.S. banking system is sound and resilient. Tighter credit conditions for households and businesses are likely to weigh on economic activity, hiring, and inflation. The extent of these effects remains uncertain. The Committee remains highly attentive to inflation risks.\nThe Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the longer run. In support of these goals, the Committee decided to maintain the target range for the federal funds rate at 5-1/4 to 5-1/2 percent. The Committee will continue to assess additional information and its implications for monetary policy. In 

In [12]:
minutes_list[-1]

'\nMinutes of the Federal Open Market Committee\n\nSeptember 19-20, 2023\nA joint meeting of the Federal Open Market Committee and the Board of Governors of the Federal Reserve System was held in the offices of the Board of Governors on Tuesday, September 19, 2023, at 10:30 a.m. and continued on Wednesday, September 20, 2023, at 9:00 a.m.1\nAttendance \r\nJerome H. Powell, Chair\r\nJohn C. Williams, Vice Chair\r\nMichael S. Barr\r\nMichelle W. Bowman\r\nLisa D. Cook\r\nAustan D. Goolsbee\r\nPatrick Harker\r\nPhilip N. Jefferson\r\nNeel Kashkari\r\nAdriana D. Kugler\r\nLorie K. Logan\r\nChristopher J. Waller\nThomas I. Barkin, Raphael W. Bostic, Mary C. Daly, and Loretta J. Mester, Alternate Members of the Committee\nSusan M. Collins and Jeffrey R. Schmid, Presidents of the Federal Reserve Banks of Boston and Kansas City, respectively\nKathleen O\'Neill Paese, Interim President of the Federal Reserve Bank of St. Louis\nJoshua Gallin, Secretary\r\nMatthew M. Luecke, Deputy Secretary\r\nB

## Prompt and Price

Usage: https://platform.openai.com/account/usage

In [13]:
default_template = 'Translate the following text into Korean:\n{}'

def build_prompt(text, template=default_template):
    prompt = template.format(text)
    return prompt

In [14]:
prompt = build_prompt(statement_list[-1])
print(prompt)

Translate the following text into Korean:

September 20, 2023
Federal Reserve issues FOMC statement
For release at 2:00 p.m. EDT                     
                
                
Share
Recent indicators suggest that economic activity has been expanding at a solid pace. Job gains have slowed in recent months but remain strong, and the unemployment rate has remained low. Inflation remains elevated.
The U.S. banking system is sound and resilient. Tighter credit conditions for households and businesses are likely to weigh on economic activity, hiring, and inflation. The extent of these effects remains uncertain. The Committee remains highly attentive to inflation risks.
The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the longer run. In support of these goals, the Committee decided to maintain the target range for the federal funds rate at 5-1/4 to 5-1/2 percent. The Committee will continue to assess additional information and its implicati

In [15]:
num_input_tokens = get_num_input_tokens(prompt)
print(num_input_tokens)

506


In [26]:
answer = query_message(prompt, verbose=True)

input tokens: 506
output tokens: 1109
input to output ratio: 2.192
price: $ 0.00298


In [17]:
print(answer)

2023년 9월 20일
연방준비제도는 FOMC 성명서를 발표합니다.
동부 표준시로 오후 2시에 공개됩니다.

최근 지표들은 경제 활동이 견실한 속도로 확장되고 있다는 것을 시사합니다. 취업 증가는 최근 몇 달 동안 둔화되었지만 여전히 강하며, 실업률은 낮은 수준을 유지하고 있습니다. 인플레이션은 여전히 높은 상태입니다.
미국 은행 시스템은 견고하고 탄력적입니다. 가계와 기업을 위한 신용 조건의 강화는 경제 활동, 고용, 그리고 인플레이션에 부담을 줄 것으로 예상됩니다. 이러한 영향의 정도는 불확실합니다. 위원회는 인플레이션 위험에 대해 매우 주의 깊게 대응하고 있습니다.
위원회는 장기적으로 최대 고용과 2%의 인플레이션을 달성하기 위해 노력하고 있습니다. 이러한 목표를 지원하기 위해 위원회는 연방 기금 금리의 목표 범위를 5-1/4%에서 5-1/2%로 유지하기로 결정했습니다. 위원회는 추가 정보와 이에 대한 통화 정책의 영향을 계속 평가할 것입니다. 인플레이션을 시간이 지나면서 2%로 되돌리기 위해 필요한 추가 정책 강화의 정도를 결정할 때, 위원회는 통화 정책의 누적 조여, 통화 정책이 경제 활동과 인플레이션에 미치는 시차, 그리고 경제와 금융 동향을 고려할 것입니다. 또한, 위원회는 이전에 발표한 계획에 따라 국채와 기관 부채, 기관 유동성을 계속 축소할 것입니다. 위원회는 인플레이션을 2% 목표로 되돌리는 데 강력하게 헌신하고 있습니다.
통화 정책의 적절한 입장을 평가함에 있어서, 위원회는 경제 전망에 대한 들어오는 정보의 영향을 계속 모니터링할 것입니다. 위원회는 위원회의 목표 달성을 방해할 수 있는 위험이 발생할 경우, 필요에 따라 통화 정책의 입장을 조정할 준비가 되어 있습니다. 위원회의 평가는 노동 시장 상황, 인플레이션 압력과 인플레이션 기대, 그리고 금융 및 국제 동향을 포함한 다양한 정보를 고려할 것입니다.
통화 정책 조치에 대한 투표는 제롬 H. 파월 의장, 존 C. 윌리엄스 부의장, 마이클 S. 바르, 미셸 W. 보우먼, 리사 D. 쿡, 

In [27]:
prompt = build_prompt(statement_list[-2])
answer = query_message(prompt, verbose=True)

input tokens: 496
output tokens: 1081
input to output ratio: 2.179
price: $ 0.00291


In [19]:
input_to_output = 2.20

In [20]:
max_total_length = 4097
max_input_length = round(max_total_length / (1+input_to_output))
print(max_input_length)

1280


In [21]:
prompt = build_prompt(minutes_list[-1])
num_input_tokens = get_num_input_tokens(prompt)
print(num_input_tokens)

8279


In [22]:
prompts = [build_prompt(statement) for statement in statement_list]

In [24]:
total_input_tokens = 0
for prompt in prompts:
    total_input_tokens += get_num_input_tokens(prompt)
print('total input tokens:', total_input_tokens)
print('estimated price: $', estimate_price(total_input_tokens, input_to_output))

total input tokens: 3011
estimated price: $ 0.01776
